This notebook will be a tutorial in how to calculate volume, as a difference between consecutive DEMs.
A rough overview of the steps to be followed is given below-
1. Sort DEMs in chronological order. 
2. Inspect DEMs and Mosaics to check for distortions. 
3. Create Vector Mask to remove Areas with distortions. 
4. Mask original DEM with created vector mask. 
5. Use grass's `r.fillnulls` command to create the gaps left by the masked distortions. 
6. Obtain set of error free DEMs.
7. Using this set, obtain delta DEMs between consecutive DEMs, or as per requirement. 
8. Do summation over obtained Delta DEM and multiply by area of a pixel, as per scale to obtain volume. 


*Note- Steps with a * \* *indicates that there are algorithms/variables to be tuned*

In [1]:
import gdal
import cv2
from subprocess import call
import numpy as np
import matplotlib.pyplot as plt
#Custom Written Scripts
try:
    import dem_proj as dp
    import dem_filtering as df
    import dem_perf_check as dpc
    from raster_chunks import GeoChunks as gc 
    
except ImportError:
    print("Import Error. Check if dem_filtering.py is present in PYTHONPATH")
    print("PYTHONPATH = ")
    call(["echo", "$PYTHONPATH"])

# Bit of formatting to change default inline code style:
from IPython.core.display import HTML
HTML("""<style> .rendered_html code { 
    padding: 2px 4px;
    color: #c7254e;
    background-color: #f9f2f4;
    border-radius: 4px;
} </style>""")

Setting up PATH variables. 

We will continue to set up PATH variables, as and when necessary.



In [2]:
PATH_PREFIX = "/home/madhavm/vimana/workdir/rough/cs_volume_2/final_proc/UTM/"
feb14_dem = PATH_PREFIX + "14feb2017_VC_dem.tif"
may09_dem = PATH_PREFIX + "09may2017_VC_dem.tif"
june16_dem = PATH_PREFIX + "16june2017_VC_dem.tif"
apr29_dem = PATH_PREFIX + "29apr2017_VC_dem.tif"
may15_dem = PATH_PREFIX + "15may2017_VC_dem.tif"
may20_dem = PATH_PREFIX + "20may2017_VC_dem.tif"
june03_dem = PATH_PREFIX + "03june2017_VC_dem.tif"

In [7]:

t1 = gc(feb14_dem)
t2 = gc(apr29_dem)
t3 = gc(may09_dem)
t4 = gc(may15_dem)
t5 = gc(may20_dem)
t6 = gc(june03_dem)
t7 = gc(june16_dem)

#Pixel Areas
err_t1 = t1.geo_trans[1] ** 2
err_t2 = t2.geo_trans[1] ** 2
err_t3 = t3.geo_trans[1] ** 2
err_t4 = t4.geo_trans[1] ** 2
err_t5 = t5.geo_trans[1] ** 2
err_t6 = t6.geo_trans[1] ** 2
err_t7 = t7.geo_trans[1] ** 2


In [9]:
m2_vol_t1 = df.volumeRefPlane(feb14_dem)
m2_vol_t2 = df.volumeRefPlane(apr29_dem)
m2_vol_t3 = df.volumeRefPlane(may09_dem)
m2_vol_t4 = df.volumeRefPlane(may15_dem)
m2_vol_t5 = df.volumeRefPlane(may20_dem)
m2_vol_t6 = df.volumeRefPlane(june03_dem)
m2_vol_t7 = df.volumeRefPlane(june16_dem)

err_vol_t1 = df.sumPixDEM(feb14_dem) * err_t1
err_vol_t2 = df.sumPixDEM(may09_dem) *err_t2
err_vol_t3 = df.sumPixDEM(june16_dem) * err_t3

print("All volumes are in cubic metres - \n")
print("Volume excavated between 14th February and 9th May - " + str(m2_vol_t1 - m2_vol_t2))
print("Error is +/- " + str(err_vol_t1))
print("Volume excavated between 14th february and 16th June - " + str(m2_vol_t1 - m2_vol_t3))
print("Error is +/- " + str(err_vol_t1 + err_vol_t3))
print("Volume excavated between session 1 and session 4 - " + str(m2_vol_t1 - m2_vol_t4))
print("Volume excavated between session 1 and session 5 - " + str(m2_vol_t1 - m2_vol_t5))
print("Volume excavated between session 1 and session 6 - " + str(m2_vol_t1 - m2_vol_t6))
print("Volume excavated between session 1 and session 6 - " + str(m2_vol_t1 - m2_vol_t7))

print("\nDelta Volumes from consecutive sessions..\n")
print("Volume excavated between 9th May and 16th June - " + str(m2_vol_t2 - m2_vol_t3))
print("Error is +/- " + str(err_vol_t3 + err_vol_t2))
print("Volume excavated between session 3 and session 4 - " + str(m2_vol_t3 - m2_vol_t4))
print("Volume excavated between session 4 and session 5 - " + str(m2_vol_t4 - m2_vol_t5))
print("Volume excavated between session 5 and session 6 - " + str(m2_vol_t5 - m2_vol_t6))
print("Volume excavated between session 5 and session 6 - " + str(m2_vol_t6 - m2_vol_t7))


All volumes are in cubic metres - 

Volume excavated between 14th February and 9th May - 1406.06399128
Error is +/- 5208.29419789
Volume excavated between 14th february and 16th June - 8282.43484556
Error is +/- 7832.76061082
Volume excavated between session 1 and session 4 - 7909.8146093
Volume excavated between session 1 and session 5 - 13494.6370977
Volume excavated between session 1 and session 6 - 16111.2501685
Volume excavated between session 1 and session 6 - 21262.6806049

Delta Volumes from consecutive sessions..

Volume excavated between 9th May and 16th June - 6876.37085428
Error is +/- 7955.49781393
Volume excavated between session 3 and session 4 - -372.620236264
Volume excavated between session 4 and session 5 - 5584.82248835
Volume excavated between session 5 and session 6 - 2616.61307084
Volume excavated between session 5 and session 6 - 5151.43043645


In [10]:
print(err_vol_t1)
print(err_vol_t2)
print(err_vol_t3)


5208.29419789
5331.031401
2624.46641293
